In [5]:
import pandas as pd
import numpy as np
import nltk
import pickle
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

In [2]:
np.random.seed(500)

with open('../data/dataset/train/us_train.text', 'r', encoding="utf8") as file:
    X_train1 = file.readlines()

with open('../data/dataset/train/us_train.labels', 'r', encoding="utf8") as file:
    y_train = file.readlines()

with open('../data/dataset/test/us_test.text', 'r', encoding="utf8") as file:
    X_test1 = file.readlines()

with open('../data/dataset/test/us_test.labels', 'r', encoding="utf8") as file:
    y_test = file.readlines()

In [ ]:
X_train1 = [tweet.lower() for tweet in X_train1]
X_train1 = [word_tokenize(tweet) for tweet in X_train1]
X_test1 = [tweet.lower() for tweet in X_test1]
X_test1 = [word_tokenize(tweet) for tweet in X_test1]

In [ ]:
X_train = []

for tweet in X_train1:
    words = []
    wnl = WordNetLemmatizer()

    for word in tweet:
        if word not in stopwords.words('english') and word.isalpha():
            word = wnl.lemmatize(word)
            words.append(word)

    X_train.append(str(words))

print(X_train[0])

In [ ]:
X_test = []

for tweet in X_test1:
    words = []
    wnl = WordNetLemmatizer()

    for word in tweet:
        if word not in stopwords.words('english') and word.isalpha():
            word = wnl.lemmatize(word)
            words.append(word)

    X_test.append(str(words))

print(X_test[0])

In [ ]:
with open('X_train', 'wb') as fp:
    pickle.dump(X_train, fp)

with open('X_test', 'wb') as fp:
    pickle.dump(X_test, fp)

In [3]:
# X_train = []
# X_test = []
with open('X_train', 'rb') as fp:
    X_train = pickle.load(fp)

with open('X_test', 'rb') as fp:
    X_test = pickle.load(fp)

In [4]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(np.concatenate((X_train,X_test)))

Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

In [ ]:
print(Tfidf_vect.vocabulary_)

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)

In [ ]:
with open('../results/naive_bayes.labels', 'w') as f:
    for pred in predictions_NB:
        f.write("%s" %pred)

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

In [ ]:
with open('../results/svm_c1_linear.labels', 'w') as f:
    for pred in predictions_SVM:
        f.write("%s" %pred)

In [12]:
Linear_SVM = make_pipeline(StandardScaler(with_mean=False), LinearSVC(C=0.05, verbose=1, tol=1e-05))

Linear_SVM.fit(Train_X_Tfidf, y_train)

predictions_SVM = Linear_SVM.predict(Test_X_Tfidf)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

[LibLinear]SVM Accuracy Score ->  21.188000000000002


In [13]:
with open('../results/svm_c05_linear.labels', 'w') as f:
    for pred in predictions_SVM:
        f.write("%s" %pred)